In [2]:
# "Design and implement a rate limiter for an LLM API service. The service needs to:

# Limit each user to 10 requests per minute
# Handle concurrent requests
# Work across multiple server instances
# Maintain accurate counts even if some requests take >1 minute to process"

In [5]:
import threading
import time
from collections import defaultdict, deque


class RateLimiter:
    def __init__(
        self, max_requests, period
    ):  # core properties of rate limit are the # of requests allowed in time period
        self.max_requests = max_requests
        self.period = period
        self.lock = (
            threading.Lock()
        )  # lock to ensure thread safety - needed to avoid race conditions with concurrent requests
        self.requests = defaultdict(
            deque
        )  # dictionary to store requests for each user, using deque to efficiently remove old requests
        # example requests - {user_id: [1716604800.0, 1716604801.0, 1716604802.0], user_id2: [1716604800.0, 1716604801.0]}
        # use deque rather than list so that we can efficiently remove old requests - O(1) time complexity for popleft()

    def is_allowed(self, user_id):  # is the operation allowed (with rate limit)
        with self.lock:  # once starts, all must happen together for thread safety
            current_time = time.time()
            if user_id in self.requests:  # if the user has made requests already
                while (
                    self.requests[user_id] and self.requests[user_id][0] <= current_time - self.period
                ):  # while the older request is older than the time period, remove it
                    self.requests[user_id].popleft()  # remove the oldest request from the user
            if (
                len(self.requests[user_id]) < self.max_requests
            ):  # if the user has made less than the max requests in the time period, allow the request
                self.requests[user_id].append(current_time)  # add the current request to the user's requests
                return True  # request is allowed
            else:
                return False  # request is denied due to rate limiting


rate_limiter = RateLimiter(5, 60)  # 100 requests per minute


def handle_request(user_id):
    if rate_limiter.is_allowed(user_id):
        print(
            f"Request from user {user_id} is allowed. Their requests: {rate_limiter.requests[user_id]} (len {len(rate_limiter.requests[user_id])} < {rate_limiter.max_requests})"
        )
    else:
        print(
            f"Request from user {user_id} is denied due to rate limiting. Their requests: {rate_limiter.requests[user_id]} (len {len(rate_limiter.requests[user_id])} >= {rate_limiter.max_requests})"
        )

In [6]:
# Simulate requests
for i in range(15):
    handle_request("user1")
    time.sleep(0.5)  # sleep 500ms between requests

Request from user user1 is allowed. Their requests: deque([1732422515.3557131]) (len 1 < 5)
Request from user user1 is allowed. Their requests: deque([1732422515.3557131, 1732422515.8608608]) (len 2 < 5)
Request from user user1 is allowed. Their requests: deque([1732422515.3557131, 1732422515.8608608, 1732422516.3658729]) (len 3 < 5)
Request from user user1 is allowed. Their requests: deque([1732422515.3557131, 1732422515.8608608, 1732422516.3658729, 1732422516.870552]) (len 4 < 5)
Request from user user1 is allowed. Their requests: deque([1732422515.3557131, 1732422515.8608608, 1732422516.3658729, 1732422516.870552, 1732422517.376595]) (len 5 < 5)
Request from user user1 is denied due to rate limiting. Their requests: deque([1732422515.3557131, 1732422515.8608608, 1732422516.3658729, 1732422516.870552, 1732422517.376595]) (len 5 >= 5)
Request from user user1 is denied due to rate limiting. Their requests: deque([1732422515.3557131, 1732422515.8608608, 1732422516.3658729, 1732422516.87